In [35]:
from dotenv import load_dotenv
load_dotenv()

True

In [36]:
import os
import json
import pandas as pd
import traceback

In [37]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [38]:
#openai_key = "sk-lH7bJCo2_JlVUxp_mnWMrdir5atw1ORS3m5fk469XRT3BlbkFJvoWgNVf9ucNoenAG5LfktYP9e5--5SDJcADQ3HPKQA"
openai_key = os.getenv("OPENAI_API_KEY")
my_llm = ChatOpenAI(openai_api_key = openai_key, model_name = "gpt-3.5-turbo-0125", temperature = 0.3)

In [39]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [40]:
Template ="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [41]:
QUIZ_Prompt = PromptTemplate(
    input_variables = ["text", "number", "subject", "tone", "response_json"],
    template = Template
)

In [42]:
Template2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [43]:
QUIZ_Evaluation_Prompt = PromptTemplate(
    input_variables = ["subject", "quiz"],
    template = Template2
)

In [50]:
QUIZ_Chain = LLMChain(
    llm=my_llm,
    prompt=QUIZ_Prompt,
    output_key="quiz", 
    verbose=True
)


In [51]:

Review_Chain = LLMChain(
    llm=my_llm,
    prompt=QUIZ_Evaluation_Prompt,
    output_key="review",  
    verbose=True
)

In [53]:

My_chain = SequentialChain(
    chains=[QUIZ_Chain, Review_Chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],  
    output_variables=["quiz", "review"],  
    verbose=True
)

In [55]:
file_path =r"C:\Users\Yassine\Desktop\MCQ_Generator\data.txt"

In [56]:
with open(file_path, 'r') as file:
    Text = file.read()

In [58]:
print(Text)

Generative artificial intelligence (generative AI, GenAI,[1] or GAI) is a subset of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data.[2] These models often generate output in response to specific prompts.[3][4] Generative AI systems learn the underlying patterns and structures of their training data, enabling them to create new data.[5][6]

Improvements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the early 2020s. These include chatbots such as ChatGPT, Copilot, Gemini and LLaMA, text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney and DALL-E, and text-to-video AI generators such as Sora.[7][8][9][10] Companies such as OpenAI, Anthropic, Microsoft, Google, and Baidu as well as numerous smaller firms have developed generative AI models.[3][11][12]

Generative AI has uses across a wide range of in

In [59]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [61]:
NUMBER = 5
SUBJECT = "generative ai"
TONE = "intermediate"

In [62]:
with get_openai_callback() as cb:
    response = My_chain(
        {
            "text" : Text,
            "number" : NUMBER,
            "subject" : SUBJECT,
            "tone" : TONE,
            "response_json" : json.dumps(RESPONSE_JSON)
        })

    

C:\Users\Yassine\AppData\Local\Temp\ipykernel_34464\3851199234.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = My_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Generative artificial intelligence (generative AI, GenAI,[1] or GAI) is a subset of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data.[2] These models often generate output in response to specific prompts.[3][4] Generative AI systems learn the underlying patterns and structures of their training data, enabling them to create new data.[5][6]

Improvements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the early 2020s. These include chatbots such as ChatGPT, Copilot, Gemini and LLaMA, text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney and DALL-E, and text-to-video AI generators such as Sora.[7][8][9][10] Companies such as OpenAI, Anthropic, Microsoft, Google, and Baidu as well as numerous sma

In [63]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1776
Prompt Tokens:1198
Completion Tokens:578
Total Cost:0.0014659999999999999


In [68]:
quiz = response.get("quiz")

In [70]:
quiz = json.loads(quiz)

In [71]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [74]:
quiz = pd.DataFrame(quiz_table_data)

In [75]:
quiz.to_csv("genAI.csv", index=False)